# Clause Parser Algorithm with Custom Metrics

In [1]:
!python --version
!python -m spacy download en_core_web_lg
print("Downloaded")

Python 3.6.5
    100% |████████████████████████████████| 852.3MB 75.9MB/s ta 0:00:01  4% |█▋                              | 42.1MB 13.8MB/s eta 0:00:59    7% |██▌                             | 67.6MB 10.8MB/s eta 0:01:13    8% |██▊                             | 71.6MB 12.2MB/s eta 0:01:04    9% |███                             | 81.0MB 12.2MB/s eta 0:01:04    10% |███▌                            | 93.2MB 12.3MB/s eta 0:01:02    11% |███▉                            | 100.9MB 14.9MB/s eta 0:00:51    15% |████▉                           | 128.9MB 73.2MB/s eta 0:00:10    16% |█████▍                          | 143.4MB 64.4MB/s eta 0:00:12    17% |█████▌                          | 146.7MB 8.5MB/s eta 0:01:24    24% |████████                        | 212.8MB 72.0MB/s eta 0:00:09    27% |█████████                       | 236.9MB 52.2MB/s eta 0:00:12    31% |██████████                      | 265.3MB 10.1MB/s eta 0:00:59    34% |███████████                     | 290.8MB 10.2MB/s eta 0:00:55    3

In [2]:
import spacy
import html
from spacy import displacy

nlp = spacy.load('en_core_web_lg')
print("Loaded models")

Loaded models


In [3]:
from random import shuffle
def print_sample_list(arr):
    indices = [i for i in range(len(arr))]
    shuffle(indices)
    for x in indices[:min(5, len(arr))]:
        print(x, arr[x])

In [4]:
from io import StringIO
import pandas as pd, numpy as np

tsv = StringIO("""survey_id	prompt_number	prompt_id	prompt	response	response_delimited
2546	36	48	Sometimes I wish that	my mom would come home more often.	|1| my mom |2| would come home more often.
2507	1	90	My family	has a cat...I don't have a cat though I have 2 dogs	|1| has a cat...|2| I don't have a cat |3| though I have 2 dogs
2510	33	33	When I am nervous	my hands get shaky	|1| my hands |2| get shaky
2509	22	43	At times I worry about	my own dreams, nightmares I have thousands of them	|1| my own dreams, nightmares |2| I have thousands of them
3356	34	47	Technology	its crazy and i wish i could keep up	|1| its crazy and |2| i wish |3| i could keep up
2537	26	26	When I get mad	I sometimes use big voice then I go somewhere quiet so I can cry by myself.	|1| I sometimes use big voice |2| then I go somewhere quiet |3| so I can cry by myself.
1869	25	25	My main problem is	a feeling that I am not entitled	|1| a feeling |2| that I am not entitled
3127	29	29	If my mother	was a different lady, I would not be who I am.	|1| was a different lady, |2| I would not be who I am.
1832	26	26	When I get mad	it takes a long time to build up, but I am slow to forgive.	|1| it takes a long time to build up, |2| but I am slow to forgive.
2390	17	17	When they avoided me	I felt safe, knowing that I could receive a warm welcome from my family and friends.	|1| I felt safe, |2| knowing |3| that I could receive a warm welcome |4| from my family and friends.
1880	36	48	Sometimes I wish that	I could be younger and have done things differently.	|1| I could be younger |2| and have done things differently.
2407	17	17	When they avoided me	I sought to find out 'why?' . Knowing the 'why' can begin Awareness and Healing	|1| I sought |2| to find out 'why?' . |3| Knowing the 'why' |4| can begin Awareness and Healing
3233	5	5	Being with other people	is both a learning experience and an opportunity to exchange views.	|1| is both a learning experience and an opportunity |2| to exchange views.
2102	25	25	My main problem is	only a main problem if I think it is.	|1| only a main problem |2| if I think |3| it is.
1529	10	10	When people are helpless	At times I try to find other ways of doing thing. But sometime I might be disappointed and stop or quit.	|1| At times I try |2| to find other ways |3| of doing thing. |4| But sometime I might be disappointed |5| and stop |6| or quit.
2856	31	31	My father	taught me how to think critically and to love deeply.	|1| taught me |2| how to think critically |3| and to love deeply.
1975	32	32	If I can\'t get what I want	I try to work out another way of getting what I want	|1| I try |2| to work out another way |3| of getting |4| what I want
2157	17	17	When they avoided me	hmmm... I\'m stumped. I have no idea how to respond to this... Although there is a sense of relief...	|1| hmmm... I'm stumped. |2| I have no idea |3| how to respond to this...|4| Although there is a sense of relief...
2409	5	73	Loving other people	can deepen with familiarity. Even with those who aren't a natural fit the differences between us can add color to the world as well as an opportunity to turn inward to see what is being triggered. Having said that, it's difficult for me to open my heart and trust.	|1| can deepen with familiarity. |2| Even with those who aren't a natural fit |3| the differences between us can add color to the world |4| as well as an opportunity to turn inward |5| to see |6| what is being triggered. |7| Having said that, |8| it's difficult for me |9| to open |10| my heart |11| and trust.
2046	10	10	When people are helpless	I am inspired to reach out and be of service.	|1| I am inspired |2| to reach out |3| and be of service.
2837	18	42	Rules	are guard rails. It is good and helpful to have them. When you head in the right direction you never get in touch with them, no matter how fast or slow you travel. Good rules are not in your way.	|1| are guard rails. |2| It is good and helpful |3| to have them. |4| When you head in the right direction |5| you never get in touch with them, |6| no matter how fast or slow you travel. |7| Good rules are not in |8| your way.
2381	34	47	Technology	is a great tool, nice to have and can bring global humanity closer together in ways never before imagined. But if it were to disappear tomorrow, I'm of the generation that would be able to survive.	|1| is a great tool, |2| nice to have |3| and can bring global humanity closer together |4| in ways never before imagined. |5| But if it were |6| to disappear tomorrow, |7| I'm of the generation |8| that would be able |9| to survive.
2344	12	12	A good boss	A good boss is critical toward making a workplace environment healthy and successful!	|1| A good boss is critical |2| toward making a workplace environment healthy and successful!
2952	25	25	My main problem is	getting over decades spent proving myself.	|1| getting over |2| decades spent proving myself.
2721	16	16	I feel sorry	for the pain that all humans and all beings on this earth must endure as part of the experience of being here.	|1| for the pain |2| that all humans and all beings on this earth must endure |3| as part of the experience of being here.
2276	30	30	If I were in charge	i would change that by facilitating change that made everyone in charge ... to the degree of their capabilities in wordly matters, but always in charge of their interior and their right to decide for themselves what is real and true for them.	|1| i would change that |2| by facilitating change |3| that made everyone in charge ...|4| to the degree of their capabilities in wordly matters,|5| but always in charge |6| of their interior and |7| their right |8| to decide for themselves |9| what is real and true for them.
3099	22	43	At times I worry about	big, big existential questions, and little microscopic ones... sometimes within milliseconds of one another.	|1| big, big existential questions, and little microscopic ones...|2| sometimes within milliseconds of one another.
1883	1	1	Raising a family	is a profound gift. I am tending the future. I am tending the life processes. It is a place for love and life and evolution to soar.	|1| is a profound gift. |2| I am tending the future. |3| I am tending the life processes. |4| It is a place for love and life and evolution |5| to soar.
1996	22	43	At times I worry about	how differently I see and process things from many of the people I encounter, and whether I am wisely acting and interacting.	|1| how differently I see |2| and process things |3| from many of the people I encounter, |4| and whether I am wisely acting |5| and interacting.
2374	11	39	What I like to do best is	be a conduit. There is no I.	|1| be a conduit. |2| There is no I.
3158	5	5	Being with other people	can be an isolating or a hugely enriching experience, the latter when each others presence liberates emergent thoughts or self expressions bringing forth a collective that is more than the sum of its parts	|1| can be an isolating or a hugely enriching experience, |1| the latter when each others presence liberates emergent thoughts or self expressions |3| bringing forth a collective |4| that is more than the sum of its parts
2690	28	83	A teacher has the right to	I don't mean to be oppositional. . but this sentence stem. . .like so many others. . isn't one I can really answer outside of context. Rights. This is a wobbly construct. I'm sorry. I'm out of time otherwise I would write a bit about the possibility of rights . .the idea of having a right. . the usage of the language of rights.	|1| I don't mean |2| to be oppositional. .|3| but this sentence stem. . .like so many others. . isn't one |4| I can really answer outside of context.|5| Rights. This is a wobbly construct.|6| I'm sorry.|7| I'm out of time |8| otherwise I would write a bit about the possibility of rights ..the idea of having a right. . the usage of the language of rights.
3321	5	5	Being with other people	is most often a pleasure and fascinating experience of seeing a completely different version of Self.	|1| is most often a pleasure and fascinating experience |2| of seeing a completely different version of Self.
2404	14	14	The past	is a construct to give form and structure to the present moment.	|1| is a construct |2| to give form and structure to the present moment.
2215	13	40	We could make the world a better place if	the mystery was okay to be as a question, that is to say if the question was okay to be a question and the answer could be a question and the answers and questions could be held together because they need each other and we realize a little faster that they won't end and they are growing together and rest in that.	|1| the mystery was okay |2| to be as a question, |3| that is to say |4| if the question was okay |5| to be a question |6| and the answer could be a question |7| and the answers and questions could be held together |8| because they need each other |9| and we realize a little faster |10| that they won't end |11| and they are growing together |12| and rest in that.
3181	12	12	A good boss	is a solar being ... one that brings light and eternal love without demanding anything of the others ... an energetic and gravitational field of such calm, peaceful magnitude that all are warmed and captivated by their glow ... brings a new burst life to all those who are in their presence	|1| is a solar being ...|2| one that brings light and eternal love |3| without demanding anything of the others ..|4| an energetic and gravitational field of such calm, peaceful magnitude |5| that all are warmed and captivated by their glow ... |6| brings a new burst life |7| to all those who are in their presence
1823	35	35	My conscience bothers me if	I'm not placing my feet on the path with an open heart, mind and spirit, tuning in to what's moving and showing up fully at the dance.	|1| I'm not placing |2| my feet on the path with an open heart, mind and spirit,|3| tuning in to |4| what's moving |5| and showing up fully at the dance.
2619	2	2	When I am criticized	I can embody my personality and feel a pang contrast to the story I tell myself inside the story we tell ourselves of what we think in the moment to be an absolute before it falls away into a deeper, clearer calm of letting go.	|1| I can embody |2| my personality |3| and feel a pang contrast |4| to the story I tell myself |5| inside the story we tell ourselves |6| of what we think in the moment |7| to be an absolute |8| before it falls away into a deeper, clearer calm |9| of letting go.
1823	6	6	The thing I like about myself is	that here I sit in the drop of this essential moment, chuckling at this question! :-D	|1| that here I sit in the drop of this essential moment, |2| chuckling at this question! :-D
1909	25	25	My main problem is	integrating emptiness with everyday life, finding meaning in meaningless given the meaningless in meaning making.	|1| integrating emptiness with everyday life, |2| finding meaning in meaningless |3| given the meaningless |4| in meaning making.
1814	25	25	My main problem is	how my narccissim gets in the way of seeing the truth of the illusion of self.	|1| how my narcissm |2| gets in the way |3| of seeing the truth of the illusion of self
1805	4	37	These days, work	Is a rainbow of consciousness manifesting through each sparkling emanation called a 'me'.	|1| Is a rainbow of consciousness |2| manifesting through each sparkling emanation |3| called a 'me'.
1665	5	5	Being with other people	Is a mutually mutating meeting of universes	|1| Is a mutually mutating meeting of universes
2182	11	39	What I like to do best is	lay back and live the window that I have becomeas if a home with no walls, the infinite moves through as if there is no solid boundary.the veil is no more and the whole field that is everywhere exists where the inside and outside have no difference.	|1| lay back |2| and live the window |3| that I have become as if a home with no walls, |4| the infinite moves through |5| as if there is no solid boundary. |6| the veil is no more |7| and the whole field that is everywhere |8| exists where the inside and outside |9| have no difference.
1823	3	3	Change is	the heartbeat of the universe... aliveness calling us forth!	|1| the heartbeat of the universe...|2| aliveness calling us forth!
2721	26	26	When I get mad	i feel adrenaline through my human body and it is fierce and i love it because it moves me and it is no different than the Spirit which moves all.	|1| i feel adrenaline |2| through my human body |3| and it is fierce |4| and i love it |5| because it moves me |6| and it is no different |7| than the Spirit which moves all.
2338	1	1	Raising a family	Captivates my whole being, just as every window on reality sees it is held in Love beyond knowing.	|1| Captivates |2| my whole being, |3| just as every window on reality sees |4| it is held in Love beyond knowing.
2721	30	30	If I were in charge	Is an expression of the love animating all creation, the simple joy that moves all from rock to life to mind to All. It comes through this glorious One in all its parts and territories, from an inaccessible Heart, within, without.	|1| Is an expression |2| of the love animating all creation, |3| the simple joy that moves all from rock to life to mind to All. |4| It comes through this glorious One in all its parts and territories, from an inaccessible Heart, within, without.
1889	15	41	Privacy	the human formconsicousness separates from the One even for a moment.	|1| the human form |2| consicousness separates from the One even for a moment.""")
df = pd.read_csv(tsv, delimiter = "\t")
print(df.columns)
df.sample(frac=1).head()

Index(['survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'response_delimited'],
      dtype='object')


,survey_id,prompt_number,prompt_id,prompt,response,response_delimited
24,2721,16,16,I feel sorry,for the pain that all humans and all beings on...,|1| for the pain |2| that all humans and all b...
14,1529,10,10,When people are helpless,At times I try to find other ways of doing thi...,|1| At times I try |2| to find other ways |3| ...
12,3233,5,5,Being with other people,is both a learning experience and an opportuni...,|1| is both a learning experience and an oppor...
16,1975,32,32,If I can't get what I want,I try to work out another way of getting what ...,|1| I try |2| to work out another way |3| of g...
27,1883,1,1,Raising a family,is a profound gift. I am tending the future. I...,|1| is a profound gift. |2| I am tending the f...


In [5]:
if "prompt" in df.columns: #Original dataset
    df['sentence'] = df.apply(lambda row : "{} {}".format(row['prompt'], row['response']), axis = 1)

df.sample(frac=1).head()

,survey_id,prompt_number,prompt_id,prompt,response,response_delimited,sentence
29,2374,11,39,What I like to do best is,be a conduit. There is no I.,|1| be a conduit. |2| There is no I.,What I like to do best is be a conduit. There ...
22,2344,12,12,A good boss,A good boss is critical toward making a workpl...,|1| A good boss is critical |2| toward making ...,A good boss A good boss is critical toward mak...
31,2690,28,83,A teacher has the right to,I don't mean to be oppositional. . but this se...,|1| I don't mean |2| to be oppositional. .|3| ...,A teacher has the right to I don't mean to be ...
48,1889,15,41,Privacy,the human formconsicousness separates from the...,|1| the human form |2| consicousness separates...,Privacy the human formconsicousness separates ...
43,2182,11,39,What I like to do best is,lay back and live the window that I have becom...,|1| lay back |2| and live the window |3| that ...,What I like to do best is lay back and live th...


In [6]:
import re, html
PATTERN = "[^a-zA-Z0-9\s]+"
rgx = re.compile(PATTERN, re.IGNORECASE)

df['preprocessed_sentence'] = df['sentence'].apply(lambda ip : re.sub('\s+', ' ', rgx.sub(' ', html.unescape(ip))))
print(df.columns)
df.sample(frac=1).head()

Index(['survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'response_delimited', 'sentence', 'preprocessed_sentence'],
      dtype='object')


,survey_id,prompt_number,prompt_id,prompt,response,response_delimited,sentence,preprocessed_sentence
19,2046,10,10,When people are helpless,I am inspired to reach out and be of service.,|1| I am inspired |2| to reach out |3| and be ...,When people are helpless I am inspired to reac...,When people are helpless I am inspired to reac...
6,1869,25,25,My main problem is,a feeling that I am not entitled,|1| a feeling |2| that I am not entitled,My main problem is a feeling that I am not ent...,My main problem is a feeling that I am not ent...
38,1823,6,6,The thing I like about myself is,that here I sit in the drop of this essential ...,|1| that here I sit in the drop of this essent...,The thing I like about myself is that here I s...,The thing I like about myself is that here I s...
20,2837,18,42,Rules,are guard rails. It is good and helpful to hav...,|1| are guard rails. |2| It is good and helpfu...,Rules are guard rails. It is good and helpful ...,Rules are guard rails It is good and helpful t...
31,2690,28,83,A teacher has the right to,I don't mean to be oppositional. . but this se...,|1| I don't mean |2| to be oppositional. .|3| ...,A teacher has the right to I don't mean to be ...,A teacher has the right to I don t mean to be ...


In [7]:
df['nlp_doc'] = df['preprocessed_sentence'].apply(lambda ip : nlp(ip))
print(df.columns)
df.sample(frac=1).head()

Index(['survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'response_delimited', 'sentence', 'preprocessed_sentence', 'nlp_doc'],
      dtype='object')


,survey_id,prompt_number,prompt_id,prompt,response,response_delimited,sentence,preprocessed_sentence,nlp_doc
29,2374,11,39,What I like to do best is,be a conduit. There is no I.,|1| be a conduit. |2| There is no I.,What I like to do best is be a conduit. There ...,What I like to do best is be a conduit There i...,"(What, I, like, to, do, best, is, be, a, condu..."
47,2721,30,30,If I were in charge,Is an expression of the love animating all cre...,|1| Is an expression |2| of the love animating...,If I were in charge Is an expression of the lo...,If I were in charge Is an expression of the lo...,"(If, I, were, in, charge, Is, an, expression, ..."
9,2390,17,17,When they avoided me,"I felt safe, knowing that I could receive a wa...","|1| I felt safe, |2| knowing |3| that I could ...","When they avoided me I felt safe, knowing that...",When they avoided me I felt safe knowing that ...,"(When, they, avoided, me, I, felt, safe, knowi..."
44,1823,3,3,Change is,the heartbeat of the universe... aliveness cal...,|1| the heartbeat of the universe...|2| aliven...,Change is the heartbeat of the universe... ali...,Change is the heartbeat of the universe aliven...,"(Change, is, the, heartbeat, of, the, universe..."
35,3181,12,12,A good boss,is a solar being ... one that brings light and...,|1| is a solar being ...|2| one that brings li...,A good boss is a solar being ... one that brin...,A good boss is a solar being one that brings l...,"(A, good, boss, is, a, solar, being, one, that..."


#### Metrics

* Total % of sentences with correct reconstructions =  0.9061 . It's actually greater than 91% since complex first clauses followed by conjunctions put the conjuction with the parent clause in the first.
* Response expected = actual verbatim : 

In [8]:
def flatten_list(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

def get_children(doc):
    if len([x for x in doc.children]) == 0:
        return [doc]
    if doc.pos_ == "VERB" and doc.dep_ not in ["xcomp", "aux"]:
        return []

    op = flatten_list([get_children(l) for l in doc.lefts]) + [doc] + flatten_list([get_children(r) for r in doc.rights])
    return op

def postprocess(tokens_arr):
    if len(tokens_arr) == 1 and ( tokens_arr[0].dep_ in ["aux", "auxpass"] or tokens_arr[0].tag_ in ["VBG"]): 
        return []
    return tokens_arr

def get_text_from_tokens(tokens_arr):
    op = ' '.join([x.text for x in tokens_arr])
    op = op.replace(" nt", "nt").replace(" '", "'")
    return op

def clause_split_by_verbs(doc):
    op = []
    for token in doc:
        if token.pos_ == "VERB":
            arr = flatten_list([get_children(l) for l in token.lefts]) + [token] + flatten_list([get_children(r) for r in token.rights])
            arr = postprocess(arr)
            op.append(arr)
    if len(op)==0:
        op.append(doc)
    return op

df['split_by_verbs_arr'] = df['nlp_doc'].apply(clause_split_by_verbs)
df.sample(frac = 1).head()

,survey_id,prompt_number,prompt_id,prompt,response,response_delimited,sentence,preprocessed_sentence,nlp_doc,split_by_verbs_arr
37,2619,2,2,When I am criticized,I can embody my personality and feel a pang co...,|1| I can embody |2| my personality |3| and fe...,When I am criticized I can embody my personali...,When I am criticized I can embody my personali...,"(When, I, am, criticized, I, can, embody, my, ...","[[], [When, I, am, criticized], [], [I, can, e..."
36,1823,35,35,My conscience bothers me if,I'm not placing my feet on the path with an op...,|1| I'm not placing |2| my feet on the path wi...,My conscience bothers me if I'm not placing my...,My conscience bothers me if I m not placing my...,"(My, conscience, bothers, me, if, I, m, not, p...","[[My, conscience, bothers, me, not, placing, m..."
16,1975,32,32,If I can't get what I want,I try to work out another way of getting what ...,|1| I try |2| to work out another way |3| of g...,If I can't get what I want I try to work out a...,If I can t get what I want I try to work out a...,"(If, I, can, t, get, what, I, want, I, try, to...","[[], [If, I, can, t], [get], [what, I, want], ..."
10,1880,36,48,Sometimes I wish that,I could be younger and have done things differ...,|1| I could be younger |2| and have done thing...,Sometimes I wish that I could be younger and h...,Sometimes I wish that I could be younger and h...,"(Sometimes, I, wish, that, I, could, be, young...","[[Sometimes, I, wish], [], [that, I, could, be..."
46,2338,1,1,Raising a family,"Captivates my whole being, just as every windo...","|1| Captivates |2| my whole being, |3| just as...","Raising a family Captivates my whole being, ju...",Raising a family Captivates my whole being jus...,"(Raising, a, family, Captivates, my, whole, be...","[[Raising, a, family], [Captivates, my, whole,..."


df postprocessing and the clause delimiting

In [9]:
df['split_by_verbs'] = df['split_by_verbs_arr'].apply(lambda op : ' '.join(["|{}| {}".format(i+1, op[i]) for i in range(len(op))]))

def remove_prompts(df):
    prompt, tokens_arr = df.prompt, df.split_by_verbs_arr
    pdoc = nlp(prompt)
    ignore_indices = [x.i for x in pdoc]
    new_arr = []
    for clause in tokens_arr:
        new_clause = [t for t in clause if t.i not in ignore_indices]
        if len(new_clause) >= 0:
            new_arr.append(new_clause)
    return [x for x in new_arr if len(x) != 0]

def process_text_df(clauses_arr):
    new_arr = []
    # first pass
    first_pass = []
    tok_arr = [[ tok.i for tok in clause] for clause in clauses_arr]

    for i in range(len(tok_arr)):
        x = tok_arr[i]
        if len(x) ==  0:
            continue
        is_subset = False
        for y in tok_arr:
            if set(x).issubset(y) and not set(x) == set(y):
                is_subset = True
        if not is_subset:
            first_pass.append(clauses_arr[i])
    
    for clauses in first_pass:
        if len(clauses) == 0:
            continue
        txt = get_text_from_tokens(clauses)
        new_arr.append(txt)
    
    return new_arr
        
ignore_df = None
if "prompt" in df.columns:
    df['clauses_doc_final'] = df[['prompt', 'split_by_verbs_arr']].apply(remove_prompts, axis = 1) 
    df['clauses_text_final'] = df['clauses_doc_final'].apply(process_text_df)
    df['split_by_verbs'] = df['clauses_text_final'].apply(lambda op : ' '.join(["|{}| {}".format(i+1, op[i]) for i in range(len(op))]))
    df['is_correct'] = df.apply(lambda x : re.sub("[a-z0-9]+", "", x['response_delimited'].strip().lower()) == re.sub("[a-z0-9]+", "", x['split_by_verbs'].strip().lower()), axis = 1)
    ignore_df = df[['response_delimited', 'split_by_verbs', 'is_correct']].sample(frac = 1).head()
else:
    df['split_by_verbs_arr_cleaned'] = df['split_by_verbs_arr'].apply(process_text_df)
    df['split_by_verbs'] = df['split_by_verbs_arr_cleaned'].apply(lambda op : ' '.join(["||{}| {}".format(i+1, op[i]) for i in range(len(op))]))
    df['tokens_set'] = df['nlp_doc'].apply(lambda doc : ";".join(["{}={}, {}, {}".format(token.text, token.tag_, token.dep_, token.pos_) for token in doc]))
    df[['tempID', 'sentence', 'split_by_verbs', 'tokens_set']].to_csv("./output.csv", index = False)
    df['reconstructed'] = df['split_by_verbs'].apply(lambda x : re.sub("\s+", " ", re.sub("\|\|\d+\|", "", x)))    
    df['is_correct'] = df.apply(lambda x : re.sub("[a-z0-9]+", "", x['preprocessed_sentence'].strip().lower()) == re.sub("[a-zA-Z0-9]+", "", x['reconstructed'].strip().lower()), axis = 1)
    ignore_df = df[['preprocessed_sentence', 'reconstructed', 'is_correct']].sample(frac=1).head()

print(df.columns)
ignore_df

Index(['survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'response_delimited', 'sentence', 'preprocessed_sentence', 'nlp_doc',
       'split_by_verbs_arr', 'split_by_verbs', 'clauses_doc_final',
       'clauses_text_final', 'is_correct'],
      dtype='object')


,response_delimited,split_by_verbs,is_correct
16,|1| I try |2| to work out another way |3| of g...,|1| I try to work out another way of |2| what ...,False
42,|1| Is a mutually mutating meeting of universes,|1| Is a meeting of universes |2| mutually mut...,False
8,"|1| it takes a long time to build up, |2| but ...",|1| it takes a long time but |2| to build up |...,False
24,|1| for the pain |2| that all humans and all b...,|1| for the pain |2| that all humans and all b...,False
18,|1| can deepen with familiarity. |2| Even with...,|1| can deepen with familiarity Even with thos...,False


In [10]:
df[['sentence', 'split_by_verbs']].sample(frac = 10/df.shape[0])

,sentence,split_by_verbs
7,"If my mother was a different lady, I would not...",|1| was a different lady |2| I would not be |3...
21,"Technology is a great tool, nice to have and c...",|1| is a great tool nice |2| to have and |3| c...
5,When I get mad I sometimes use big voice then ...,|1| I sometimes use big voice |2| then I go so...
20,Rules are guard rails. It is good and helpful ...,|1| are guard rails |2| It is good and helpful...
22,A good boss A good boss is critical toward mak...,|1| A good boss is critical toward |2| making ...
29,What I like to do best is be a conduit. There ...,|1| be a conduit |2| There is no I
42,Being with other people Is a mutually mutating...,|1| Is a meeting of universes |2| mutually mut...
2,When I am nervous my hands get shaky,|1| my hands get shaky
44,Change is the heartbeat of the universe... ali...,|1| the heartbeat of the universe aliveness |2...
11,When they avoided me I sought to find out 'why...,|1| I sought to find out |2| why Knowing |3| t...


In [11]:
print("Ignorable metric sentence reformation", df[df.is_correct].shape[0]/df.shape[0])

Ignorable metric sentence reformation 0.02040816326530612


In [12]:
a_poss, p_yn, p_beverb, p_get, a_def, undef = "A_pron_x", "P_yn", "P_bevb_x", "P_get_x", "A_def", "Undefined"

def voice_rule_engine(clause):
    if True not in [x.pos_ == "VERB" for x in clause]:
        return undef
    
    for x in clause:
        if x.dep_ == "poss":
            return a_poss
        
    ct = 0
    for x in clause:
        if x.text.lower().strip() in ['yes', 'no']:
            ct += 1
    if ct >= len(clause)/2:
        return p_yn

    BEING_VERBS = ['is', 'was', 'am', 'are', 'were', 'wasn', 'weren', 'isn']
    for x in clause:
        if x.text.lower().strip() in BEING_VERBS:
            return p_beverb
    
    for x in clause:
        if x.dep_ == "acomp":
            return p_get
    
    return a_def
    
def clauses_voice(arr_of_clauses):
    op = []
    for clause in arr_of_clauses:
        voice = voice_rule_engine(clause)
        op.append(voice)         
    return op

df['voice'] = df.clauses_doc_final.apply(clauses_voice)
df[['sentence', 'clauses_doc_final', 'voice']].head(20)

,sentence,clauses_doc_final,voice
0,Sometimes I wish that my mom would come home m...,"[[my, mom, would, come, home, more, often]]",[A_pron_x]
1,My family has a cat...I don't have a cat thoug...,"[[has, a, cat], [I, don, t], [have, a, cat], [...","[A_def, A_def, A_def, A_def]"
2,When I am nervous my hands get shaky,"[[my, hands, get, shaky]]",[A_pron_x]
3,"At times I worry about my own dreams, nightmar...","[[my, own, dreams, nightmares], [I, have, thou...","[Undefined, A_def]"
4,Technology its crazy and i wish i could keep up,"[[and, i, wish], [i, could, keep, up]]","[A_def, A_def]"
5,When I get mad I sometimes use big voice then ...,"[[I, sometimes, use, big, voice], [then, I, go...","[A_def, P_get_x, A_def]"
6,My main problem is a feeling that I am not ent...,"[[a, feeling], [that, I, am, not, entitled]]","[Undefined, P_bevb_x]"
7,"If my mother was a different lady, I would not...","[[was, a, different, lady], [I, would, not, be...","[P_bevb_x, A_def, P_bevb_x]"
8,When I get mad it takes a long time to build u...,"[[it, takes, a, long, time, but], [to, build, ...","[A_def, A_def, P_bevb_x, A_def]"
9,"When they avoided me I felt safe, knowing that...","[[I, felt, safe], [that, I, could, receive, a,...","[P_get_x, A_pron_x]"


In [13]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> wordnet
      Package wordnet is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Package stopwords is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [32]:
from nltk.corpus import wordnet as wn, stopwords
from decimal import Decimal
ignore_words = list(set(stopwords.words('english')))
ignore_words = ignore_words + ['keep']

DEBUG_ABSTRACTION_HIERARCHY = False
class GenericTree:
    def __init__(self, nltk_word, print_pad = ""):
        self.word = nltk_word
        self.print_pad = print_pad

    def get_max_depth(self):
        if len(self.children) == 0:
            return 0
        return max([tree.get_max_depth() for tree in self.children ])+1
    
    def print_tree(self):
        print("{}>{}".format(self.print_pad, self.word.lemma_names()[0]))
        for tree in self.children:
            tree.print_tree()
            
class HypernymTree(GenericTree):
    def __init__(self, nltk_word, pos, print_pad = ""):
        super().__init__(nltk_word, print_pad)
        if len(nltk_word.hypernyms()) == 0:
            self.children = [] 
        else:
            hyper = nltk_word.hypernyms() 
            hyper = [x for x in hyper if x.pos() == pos]
            self.children = [HypernymTree(x, pos, print_pad = "{}==".format(self.print_pad)) for x in hyper]
        
class HyponymTree(GenericTree):
    def __init__(self, nltk_word, pos, print_pad = ""): 
        super().__init__(nltk_word, print_pad)
        if len(nltk_word.hyponyms()) == 0:
            self.children = [] 
        else:
            hypo = nltk_word.hyponyms()
            hypo = [x for x in hypo if x.pos() == pos]
            self.children = [HyponymTree(x, pos, print_pad = "{}==".format(self.print_pad)) for x in hypo]

def hypernym_hierarchy(word, pos):
    toks = wn.synsets(word)
    if len(toks) == 0 or word in ignore_words:
        return 0
    hyper_trees = [HypernymTree(tok, pos) for tok in toks]
    hypers = [tree.get_max_depth() for tree in hyper_trees]
    hypo_trees = [HyponymTree(tok, pos) for tok in toks]
    hypos = [tree.get_max_depth() for tree in hypo_trees]
    if DEBUG_ABSTRACTION_HIERARCHY:
        for tree in hyper_trees:
            print("#####################################################3")
            tree.print_tree()
        for tree in hypo_trees:
            print("#####################################################3")
            tree.print_tree()

    assert len(hypers) == len(hypos)
    op = [hypos[i]*1./(hypers[i] + hypos[i]) if hypers[i]>0 or hypos[i]>0 else 0 for i in range(len(hypers))]
    #print(op)
    return max(op)

SPACY_NLTK_POS_MAPPING = {"VERB" : "v", "NOUN" : 'n', "ADJ" : "a", "ADV" : "r"}
def hypernym_score(token):
    if token.pos_ not in SPACY_NLTK_POS_MAPPING.keys():
        return 0.0
    op = hypernym_hierarchy(token.text.lower(), SPACY_NLTK_POS_MAPPING[token.pos_])
    return round(Decimal(op), 2)
    
def score_abstraction(clauses):
    op = []
    for clause in clauses:
        op.append(max([hypernym_score(x) for x in clause]))
    return op
df['abstraction_score'] = df.clauses_doc_final.apply(score_abstraction)
df[['sentence', 'clauses_doc_final', 'abstraction_score']].sample(frac=1).head(20)

,sentence,clauses_doc_final,abstraction_score
4,Technology its crazy and i wish i could keep up,"[[and, i, wish], [i, could, keep, up]]","[0.50, 0.0]"
11,When they avoided me I sought to find out 'why...,"[[I, sought, to, find, out], [to, find, out], ...","[1.00, 1.00, 1.00, 1.00]"
43,What I like to do best is lay back and live th...,"[[lay, back, and], [live, the, window, if, a, ...","[1.00, 1.00, 0.00, 0.45, 0.25, 0.00, 1.00, 0.56]"
14,When people are helpless At times I try to fin...,"[[At, times], [I, try, to, find, other, ways, ...","[0.57, 1.00, 1.00, 0.83, 0.33, 1.00, 1.00]"
10,Sometimes I wish that I could be younger and h...,"[[I, could, be, younger, and], [have, done, th...","[0.0, 1.00]"
38,The thing I like about myself is that here I s...,"[[D], [that, here, I, sit, in, the, drop, of, ...","[0.00, 1.00, 0.38]"
40,My main problem is how my narccissim gets in t...,"[[how, my, narccissim, gets, in, the, way, of]...","[1.00, 1.00]"
18,Loving other people can deepen with familiarit...,"[[can, deepen, with, familiarity, Even, with, ...","[0.56, 0.00, 0.00, 0.67, 0.86, 1.00, 0.00, 1.0..."
31,A teacher has the right to I don't mean to be ...,"[[but], [I, don, t, mean, to, be, oppositional...","[0.0, 1.00, 0.0, 0.22, 0.33, 0.54, 0.0, 0.57, ..."
23,My main problem is getting over decades spent ...,"[[getting, over, decades], [spent, proving, my...","[1.00, 1.00, 0.75]"


In [ ]:
def htmlise(df):
    html_fs = """
    <html>
        <head>
            <title>{}</title>
        </head>
        <body>
            <div>{}</div>
            <div>{}</div>
            <div>{}</div>
        </body>
    </html>"""
    op = spacy.displacy.render(df.nlp_doc, style='dep')
    with open("./html/file_{}.html".format(df.idx), "w") as f:
        f.write(html_fs.format(df.prompt, df.response, df.clauses_text_final, op))
    return
        
df['idx'] = df.index
df.apply(htmlise, axis = 1)
print("HTML processing done")